In [57]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random

random.seed(1)
np.random.seed(1)
tf.random.set_seed(1)

In [58]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = tf.cast(x_train, tf.float32), tf.cast(x_test, tf.float32)

print("x_train.shape :",x_train.shape)
print("x_test.shape :",x_test.shape)
print("y_train.shape :",y_train.shape)
print("y_test.shape :",y_test.shape)

x_train.shape : (60000, 28, 28)
x_test.shape : (10000, 28, 28)
y_train.shape : (60000,)
y_test.shape : (10000,)


In [39]:
# plt.imshow(x_train[random.randint(0, x_train.shape[0])], cmap='gray')
# plt.show

In [37]:
# print(x_train[0])

In [38]:
# print(y_train)

In [59]:
num_classes = 10
num_features = x_train.shape[1] * x_train.shape[2]

x_train, x_test = tf.reshape(x_train, [-1, num_features]), tf.reshape(x_test, [-1, num_features])

x_train, x_test = x_train/255., x_test/255.

In [67]:
W = tf.Variable(tf.random_normal_initializer(stddev=1)([num_features, num_classes]))
b = tf.Variable(tf.random_normal_initializer(stddev=1)([num_classes]))

def softmax_transformation(x):
    return tf.nn.softmax(tf.matmul(x, W) + b)

def cross_entropy(y_pred, y_true):
    y_true = tf.one_hot(y_true, depth=num_classes) # depth = vector의 크기지정
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred),1)) # reduce_sum 전/후 shape [60000,10] => [60000]
                                                                           # maximum한 값에 마이너스를 취해서 minimum을 구함(최소화하는 방향으로 W학습)
def accuracy(y_pred, y_true):
    correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [46]:
# arr1 = ['W','b']
# ex_grad = ['W_gradient', 'b_gradient']

# for zip_arr in zip(ex_grad, arr1):
#     grad, param = zip_arr
#     print(grad, "applied to", param)

W_gradient applied to W
b_gradient applied to b


In [61]:
lr = 0.01
optimizer = tf.optimizers.SGD(lr)

def run_optimization(x,y):
    with tf.GradientTape() as g:
        pred = softmax_transformation(x)
        loss = cross_entropy(pred, y)
        
    gradients = g.gradient(loss, [W,b])
    optimizer.apply_gradients(zip(gradients, [W,b]))

In [53]:
batch_size = 200
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.shuffle(60000).batch(batch_size).prefetch(1) 

# shuffle = batch로 data를 넣어줄때 섞어서
# 0...0 / 1...1 / 2...2 / 3...3 / ..... / 9...9
# 한가지 class만 가지고 오는 상황 방지
# 전체 데이터의 대표가 될 수 있도록 shuffle
# prefetch = 내가 배치 한개를 준비할 동안 다음 배치를 준비하는 것
#            속도향상(1:true / 0:false)

In [51]:
# print(train_data) # train_data가 어떤 형태인지 확인해보기
# list_data = list(train_data.as_numpy_iterator()) # x_train과 y_train을 tuple로 가지고 있다.
# n_data = len(list_data)
# print(n_data)
# print(type(list_data[0]))
# tmp1, tmp2 = list_data[0]
# print(tmp1.shape) # 28 * 28
# print(tmp2)

<TensorSliceDataset shapes: ((784,), ()), types: (tf.float32, tf.uint8)>
60000
<class 'tuple'>
(784,)
5


In [68]:
num_epoch = 4 # batch 학습을 4번 반복
display_step = 100

for epoch in range(num_epoch):
    for step, (batch_x, batch_y) in enumerate(train_data, 1):
        run_optimization(batch_x, batch_y)
        
        if step % display_step == 0:
            pred = softmax_transformation(batch_x)
            loss = cross_entropy(pred, batch_y)
            acc = accuracy(pred, batch_y)
            print("epoch:", epoch, " step:", step, " loss:", loss.numpy(), " acc:",acc.numpy())

epoch: 0  step: 100  loss: 10.480498  acc: 0.085
epoch: 0  step: 200  loss: 9.015714  acc: 0.16
epoch: 0  step: 300  loss: 8.118896  acc: 0.165
epoch: 1  step: 100  loss: 6.9010105  acc: 0.22
epoch: 1  step: 200  loss: 6.2044287  acc: 0.255
epoch: 1  step: 300  loss: 5.409092  acc: 0.3
epoch: 2  step: 100  loss: 5.233014  acc: 0.38
epoch: 2  step: 200  loss: 4.845467  acc: 0.38
epoch: 2  step: 300  loss: 4.655401  acc: 0.405
epoch: 3  step: 100  loss: 4.012035  acc: 0.44
epoch: 3  step: 200  loss: 3.3825128  acc: 0.54
epoch: 3  step: 300  loss: 3.58577  acc: 0.505


In [69]:
pred = softmax_transformation(x_test)
acc = accuracy(pred, y_test)
print(acc)

tf.Tensor(0.4863, shape=(), dtype=float32)


In [74]:
print(W)

<tf.Variable 'Variable:0' shape=(784, 10) dtype=float32, numpy=
array([[ 0.9868413 ,  0.57056284,  0.17946035, ..., -0.24785912,
        -1.4076422 ,  0.27217543],
       [ 0.5191802 ,  0.97026527,  1.3685018 , ..., -0.08326844,
        -0.7141415 , -0.5774472 ],
       [ 0.32814404,  1.41797   , -0.14009246, ..., -1.5747434 ,
        -0.21524012,  1.3278321 ],
       ...,
       [ 2.0099723 ,  1.1470464 ,  0.32427922, ..., -1.499564  ,
        -0.47324336, -1.506559  ],
       [ 0.10750776, -1.2591915 ,  1.0535144 , ...,  1.618265  ,
        -0.36039573, -2.1091177 ],
       [-0.05187156,  1.4594634 , -0.49061814, ..., -0.3647408 ,
         0.6757275 , -0.81985784]], dtype=float32)>
